# Compiling the different versions of Micro-Aevol 2

In [1]:
import os
import subprocess

KOKKOS_LIB_CMAKE = "Kokkos_DIR={0}/lib/cmake/Kokkos".format(os.environ.get("KOKKOS_INSTALL_DIR"))
print(KOKKOS_LIB_CMAKE)
print(os.getenv("KOKKOS_INSTALL_DIR"))

Kokkos_DIR=/home/onyr/Kokkos/kokkos_install/lib/cmake/Kokkos
/home/onyr/Kokkos/kokkos_install


In [2]:
# parameters

experiment_params = {
    "dna_sizes": [500, 5000, 50000], # size of DNA
    "mutation_rates": [0.0001, 0.00001, 0.000001], # mutation rate
    "world_sizes": [32, 64, 128, 256, 1024], # size of the world
}

# get project names from dirs in ./aevols directory
project_names = [d for d in os.listdir("./aevols") if os.path.isdir(os.path.join("./aevols", d))]
print(project_names)

['aevol_0']


## Compiling & Run executables

In [22]:
for project_name in project_names:
    # create directory build
    build_dir = "./aevols/" + project_name + "/build"
    if not os.path.exists(build_dir):
        os.makedirs(build_dir)
    else:
        print("build directory already exists")
        print("removing previous build directory")
        subprocess.run(["rm", "-rf", build_dir])
    
    # run cmake to generate makefiles
    subprocess.run(["cmake", "-B", f"./aevols/{project_name}/build", "-S", f"./aevols/{project_name}", "-D", KOKKOS_LIB_CMAKE])

    # run make to compile
    subprocess.run(["make", "-C", f"./aevols/{project_name}/build"])

    # for dna_size in experiment_params["dna_sizes"]:
    #     for mutation_rate in experiment_params["mutation_rates"]:
    #         for world_size in experiment_params["world_sizes"]:
    #             print("Running {} (exec_type: {}) with dna_size={}, mutation_rate={}, world_size={}".format(project_name, exec_type, dna_size, mutation_rate, world_size))
                
    #             # run the executable
    #             executable_name = "micro_aevol_cpu" if (exec_type == "nameCpu") else "micro_aevol_cuda"
    #             params = f" -w {world_size} -h {world_size} -g {dna_size} -m {mutation_rate}"
    #             subprocess.run(["./aevols/" + project_name + "/build/" + executable_name, params])

build directory already exists
removing previous build directory
-- The C compiler identification is GNU 11.3.0
-- The CXX compiler identification is GNU 11.3.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done


CMAKE_PREFIX_PATH = /home/onyr/Kokkos/kokkos_install:/home/onyr/Kokkos/kokkos_install/lib/CMake/Kokkos:/home/onyr/Kokkos/kokkos_install/lib/cmake/Kokkos
CMake Warning at /home/onyr/Kokkos/kokkos_install/lib/cmake/Kokkos/KokkosConfigCommon.cmake:35 (MESSAGE):
  The installed Kokkos configuration does not support CXX extensions.
  Forcing -DCMAKE_CXX_EXTENSIONS=Off
Call Stack (most recent call first):
  /home/onyr/Kokkos/kokkos_install/lib/cmake/Kokkos/KokkosConfig.cmake:56 (INCLUDE)
  CMakeLists.txt:44 (find_package)




-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.so (found version "1.2.11") 
-- Enabled Kokkos devices: OPENMP;CUDA;SERIAL
-- kokkos_launch_compiler is enabled globally. C++ compiler commands with -DKOKKOS_DEPENDENCE will be redirected to the appropriate compiler for Kokkos
-- Configuring done
-- Generating done
-- Build files have been written to: /home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build
make: Entering directory '/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build'
make[1]: Entering directory '/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build'
make[2]: Entering directory '/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build'
make[2]: Leaving directory '/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build'
make[2]: Entering directory '/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build'
[  8%] Building CXX object CMakeFiles/micro_aevol.dir/Abstract_ExpManager.cpp.o
[ 16%] Building C

/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/Dna.cpp(86): warning #177-D: variable "duplicate_segment" was declared but never referenced

/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/Dna.cpp(88): warning #177-D: variable "seg_length" was declared but never referenced



[ 83%] Linking CXX static library libmicro_aevol.a
make[2]: Leaving directory '/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build'
[ 83%] Built target micro_aevol
make[2]: Entering directory '/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build'
make[2]: Leaving directory '/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build'
make[2]: Entering directory '/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build'
[ 91%] Building CXX object CMakeFiles/micro_aevol_kokkos.dir/main.cpp.o
[100%] Linking CXX executable micro_aevol_kokkos
make[2]: Leaving directory '/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build'
make[1]: Leaving directory '/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build'
make: Leaving directory '/home/onyr/Documents/code/OT5_micro-aevol2/src/aevols/aevol_0/build'


/usr/bin/ld: CMakeFiles/micro_aevol_kokkos.dir/main.cpp.o: in function `main':
main.cpp:(.text+0x4d9): undefined reference to `ExpManager::ExpManager(int, int, int, double, int, int)'
/usr/bin/ld: main.cpp:(.text+0x508): undefined reference to `ExpManager::ExpManager(int)'
collect2: error: ld returned 1 exit status
make[2]: *** [CMakeFiles/micro_aevol_kokkos.dir/build.make:97: micro_aevol_kokkos] Error 1
make[1]: *** [CMakeFiles/Makefile2:111: CMakeFiles/micro_aevol_kokkos.dir/all] Error 2
make: *** [Makefile:91: all] Error 2
